<a href="https://colab.research.google.com/github/wangyouzhong/MLStudy/blob/master/Power_Prediction_MultiTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [1]:
import io
import pandas as pd

from google.colab import files

import numpy as np
from datetime import date, timedelta

In [2]:
# add daily sum and time bucket ratio to daily sum as features
def data_preprocess(df):
  df.columns = ['Id', 'Date'] + [f'TB{i}' for i in range(1, 97)]
  df['Day_Sum'] = df.apply(lambda row: np.sum(row[2:]), axis=1)
  for i in range(1, 97):
    df[f'TB{i}_Ratio'] = df.apply(lambda row: row[f'TB{i}'] / row['Day_Sum'], axis=1)
  return df

def data_shift_days(df, days_delta):
  df['Date'] = df['Date'] + pd.Timedelta(days=days_delta)
  return df

In [3]:
uploaded = files.upload()

Saving 131151000000000965.csv to 131151000000000965.csv


In [4]:
df_l1 = pd.read_csv(io.BytesIO(uploaded['131151000000000965.csv']), header=None)
df_l1 = data_preprocess(df_l1)
df_l1

,Id,Date,TB1,TB2,TB3,TB4,TB5,TB6,TB7,TB8,TB9,TB10,TB11,TB12,TB13,TB14,TB15,TB16,TB17,TB18,TB19,TB20,TB21,TB22,TB23,TB24,TB25,TB26,TB27,TB28,TB29,TB30,TB31,TB32,TB33,TB34,TB35,TB36,TB37,TB38,...,TB57_Ratio,TB58_Ratio,TB59_Ratio,TB60_Ratio,TB61_Ratio,TB62_Ratio,TB63_Ratio,TB64_Ratio,TB65_Ratio,TB66_Ratio,TB67_Ratio,TB68_Ratio,TB69_Ratio,TB70_Ratio,TB71_Ratio,TB72_Ratio,TB73_Ratio,TB74_Ratio,TB75_Ratio,TB76_Ratio,TB77_Ratio,TB78_Ratio,TB79_Ratio,TB80_Ratio,TB81_Ratio,TB82_Ratio,TB83_Ratio,TB84_Ratio,TB85_Ratio,TB86_Ratio,TB87_Ratio,TB88_Ratio,TB89_Ratio,TB90_Ratio,TB91_Ratio,TB92_Ratio,TB93_Ratio,TB94_Ratio,TB95_Ratio,TB96_Ratio
0,131151000000000965,2019-01-01,6443.213757,6339.668505,6061.736719,5927.823626,5639.515694,5508.766597,5316.491736,5380.208712,5031.950200,5025.861779,4997.452673,5103.937301,4783.836779,4832.434695,4656.268152,4739.768857,4670.468852,4689.179001,4836.636411,4621.116499,4676.205351,4746.822644,4674.089757,4748.315526,4770.425894,5060.843908,5198.606405,5452.185223,5630.595512,5649.892755,6083.688899,6407.585852,6937.257085,7556.494095,7962.109581,8535.704714,9391.133595,10203.874188,...,0.013734,0.013738,0.013910,0.013858,0.013806,0.013907,0.014100,0.014101,0.013890,0.013997,0.013796,0.013349,0.013738,0.013757,0.014048,0.013880,0.014131,0.014293,0.014259,0.014782,0.014474,0.014136,0.014734,0.014504,0.014543,0.014667,0.015059,0.014350,0.014001,0.014201,0.013762,0.012837,0.011108,0.010243,0.009444,0.008673,0.008391,0.008126,0.007645,0.007307
1,131151000000000965,2019-01-02,6420.070735,6159.470428,5902.046334,5912.334804,5612.788344,5443.938055,5232.428793,5269.273807,5005.868278,4972.147160,4921.269620,5060.840850,4749.933665,4685.472549,4599.818089,4643.062326,4584.296414,4625.539859,4790.562452,4621.935890,4568.799129,4663.123521,4592.185466,4680.662789,4714.732561,4958.732659,5141.139805,5397.173852,5524.925636,5570.967859,5985.648492,6378.632212,6901.530365,7460.776053,7836.412287,8392.216686,9238.686600,10061.429916,...,0.013830,0.013719,0.013927,0.014032,0.014011,0.013841,0.014018,0.014095,0.013945,0.013951,0.013728,0.013382,0.013656,0.013651,0.014056,0.013981,0.014383,0.014311,0.014266,0.014499,0.014569,0.014041,0.014564,0.014574,0.014590,0.014577,0.015015,0.014453,0.014120,0.014214,0.013730,0.012976,0.011126,0.010289,0.009543,0.008661,0.008455,0.008179,0.007681,0.007365
2,131151000000000965,2019-01-03,6020.718906,5928.042346,5677.283981,5601.382808,5377.565376,5175.709286,4957.847887,5079.748909,4763.842930,4793.673425,4681.720531,4811.449393,4571.442143,4464.729430,4420.783510,4465.832158,4365.294295,4440.332626,4548.564671,4358.165908,4424.041235,4466.324146,4361.214556,4460.774421,4482.159051,4691.892720,4899.219378,5086.718509,5220.855066,5359.628493,5761.974275,6043.349767,6566.527046,6986.406602,7403.576768,8038.583335,8786.628991,9601.203647,...,0.013794,0.013645,0.013841,0.013981,0.013921,0.013966,0.014028,0.013898,0.013822,0.014034,0.013904,0.013412,0.013754,0.013661,0.013830,0.013987,0.014301,0.014287,0.014228,0.014546,0.014456,0.014155,0.014610,0.014503,0.014556,0.014738,0.015122,0.014481,0.014110,0.013967,0.013610,0.012867,0.011151,0.010284,0.009509,0.008758,0.008413,0.008173,0.007712,0.007334
3,131151000000000965,2019-01-04,6342.713285,6156.494852,5925.388196,5865.362370,5590.985021,5381.203993,5191.810029,5298.470919,4988.903983,4974.633462,4975.443436,5093.157973,4691.661945,4703.524029,4586.497132,4583.323665,4568.442918,4590.523509,4772.311086,4539.198872,4571.561594,4663.919072,4558.323979,4675.140663,4711.150449,4924.799387,5028.210001,5317.970369,5481.806522,5554.154846,6000.158311,6357.582975,6849.557466,7339.592980,7775.399651,8271.971729,9186.893023,9966.620744,...,0.013785,0.013645,0.013851,0.014006,0.014003,0.013717,0.013907,0.013898,0.013868,0.014007,0.013727,0.013522,0.013742,0.013730,0.013974,0.013886,0.014340,0.014468,0.014443,0.014678,0.014586,0.014195,0.014810,0.014581,0.014654,0.014553,0.015056,0.014275,0.013990,0.014031,0.013735,0.012886,0.011026,0.010289,0.009488,0

In [ ]:
uploaded = files.upload()

Saving L3.csv to L3.csv


In [ ]:
df_l3 = pd.read_csv(io.BytesIO(uploaded['L3.csv']))
df_l3 = data_preprocess(df_l3)

In [ ]:
def build_train_data(df1, df3, start_ds, end_ds, shift_days):
  train_data = pd.DataFrame.copy(df3[(df3['Date']>=start_ds) & (df3['Date']<=end_ds)])
  
  # Join L3 data with L1 previous days data
  for i in range(1, shift_days+1):
    df_shift = pd.DataFrame.copy(df1[(df1['Date']>=(start_ds + timedelta(days=-i))) & (df1['Date']<=(end_ds + timedelta(days=-i)))])
    df_shift = data_shift_days(df_shift, i).drop(columns=['Id']).set_index('Date')
    train_data = train_data.join(df_shift, rsuffixstr=f'_d{i}_L1')
  
  # Join L3 data with L3 previous days data
  train_data = train_data.reset_index().set_index(['Date', 'Id'])
  for i in range(1, shift_days+1):
    df_shift = pd.DataFrame.copy(df3[(df3['Date']>=(start_ds + timedelta(days=-i))) & (df3['Date']<=(end_ds + timedelta(days=-i)))])
    df_shift = data_shift_days(df_shift, i).set_index(['Date', 'Id'])
    train_data = train_data.join(df_shift, rsuffixstr=f'_d{i}_L1')
  
  # Add id and week of days as one hot features
  train_data = train_data.reset_index()
  train_data = pd.concat(
      [
       train_data, 
       pd.get_dummies(train_data['Id'], prefix='Id'),
       pd.get_dummies(train_data.apply(lambda row: row['Date'].weekday(), axis=1), prefix='Weekday')
       ],axis=1)

  return train_data

In [ ]:
TRAIN_DS_START = date(2019, 1, 8)
TRAIN_DS_END = date(2019, 3, 15)
VALIDATE_DS_START = date(2019, 3, 16)
VALIDATE_DS_END = date(2019, 3, 31)

NUM_PREVIOUS_DAYS = 7

In [ ]:
train_data = build_train_data(df_l1, df_l3, TRAIN_DS_START, TRAIN_DS_END, NUM_PREVIOUS_DAYS)

In [ ]:
validate_data = build_train_data(df_l1, df_l3, VALIDATE_DS_START, VALIDATE_DS_END, NUM_PREVIOUS_DAYS)
validate_x, validate_y = validate_data.values[:,:num_features].astype('float32'), validate_data.values[:,num_features].astype('float32')

# Build A DNN Model 

In [ ]:
import tensorflow as tf 


In [ ]:
model = tf.keras.Sequential(
    layers = [
        tf.keras.layers.Dense(512, activation='relu', input_shape=(num_features,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1),
    ]
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               786944    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.mean_squared_error, metrics=['mae'])

In [ ]:
model.fit(x=train_x, y=train_y, batch_size=100, epochs = 10, validation_data = (validate_x, validate_y))

Epoch 1/10
2581/2581 [==============================] - 9s 4ms/step - loss: 29014.2520 - mae: 67.2087 - val_loss: 53874.2656 - val_mae: 118.7374
Epoch 2/10
2581/2581 [==============================] - 9s 3ms/step - loss: 21203.6230 - mae: 61.4643 - val_loss: 40326.7656 - val_mae: 108.5685
Epoch 3/10
2581/2581 [==============================] - 8s 3ms/step - loss: 20198.0566 - mae: 59.3313 - val_loss: 52244.0156 - val_mae: 97.7320
Epoch 4/10
2581/2581 [==============================] - 8s 3ms/step - loss: 18910.1914 - mae: 57.4993 - val_loss: 86573.2578 - val_mae: 139.4962
Epoch 5/10
2581/2581 [==============================] - 8s 3ms/step - loss: 19802.5234 - mae: 57.4102 - val_loss: 80913.5938 - val_mae: 109.0491
Epoch 6/10
2581/2581 [==============================] - 8s 3ms/step - loss: 18469.8496 - mae: 55.1614 - val_loss: 93480.5312 - val_mae: 124.6829
Epoch 7/10
2581/2581 [==============================] - 8s 3ms/step - loss: 17892.5234 - mae: 54.7183 - val_loss: 98728.8828 - val_

# Evaluate the Model


In [ ]:
# Model prediction
predicted_validate_y = model.predict(validate_x, batch_size = 100).flatten()
if 'predicted_value' in validate_data.columns:
  validate_data = validate_data.drop(columns=['predicted_value'])
validate_data.insert(loc = 0, column='predicted_value', value = predicted_validate_y)


In [ ]:
# Reduce to Date-Id level
validate_data_groupby = validate_data.groupby(['Date', 'Id'])
reduced_validated_data = validate_data_groupby.apply(
    lambda x: np.sum(x['TB1_L3']*validate_data['predicted_value'])).to_frame(
        name='Predicted_TB1_L3').reset_index().set_index(['Date', 'Id'])
for i in range(2, 97):
  print(f'Join predicted value {i}')
  reduced_validated_data = reduced_validated_data.join(
      validate_data_groupby.apply(
        lambda x: np.sum(x[f'TB{i}_L3']*validate_data['predicted_value'])).to_frame(
            name=f'Predicted_TB{i}_L3').reset_index().set_index(['Date', 'Id'])
  )
for i in range(1, 97):
  print(f'Join raw value {i}')
  reduced_validated_data = reduced_validated_data.join(
      validate_data_groupby.apply(
        lambda x: np.sum(x[f'TB{i}_L3']*validate_data['label'])).to_frame(
            name=f'TB{i}_L3').reset_index().set_index(['Date', 'Id'])
  )
reduced_validated_data

Join predicted value 2
Join predicted value 3
Join predicted value 4


In [ ]:
reduced_validated_data.iloc[0]

In [ ]:
import matplotlib.pyplot as plt
x = list(range(1, 97))
row0 = reduced_validated_data.iloc[20]
y0 = [row0[f'TB{i}_L3'] for i in x]
y1 = [row0[f'Predicted_TB{i}_L3'] for i in x]
plt.plot(x, y0, 'b')
plt.plot(x, y1, 'r')
plt.show()


In [ ]:
reduced_validated_data